<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/SECOP_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SECOP II

# 1. Instalaciones
# Solo ejecutar si no tienes las librerías instaladas
!pip install pandas requests

import pandas as pd
import requests
from datetime import datetime
import time

def buscar_contratos(limite=100, **filtros):
    """
    Busca contratos en SECOP II con filtros personalizables
    """
    url = "https://www.datos.gov.co/resource/rpmr-utcd.json"
    #Alternativa SODA 2.1 ilimitada: https://www.datos.gov.co/api/v3/views/rpmr-utcd/query.json

    # Construir parámetros de consulta
    params = {"$limit": limite}

    # Mapeo de nombres de parámetros (ajustado a las columnas reales del SECOP)
    mapeo_campos = {
        'nivel_entidad': 'nivel_entidad',
        'codigo_entidad': 'codigo_entidad_en_secop',
        'nombre_entidad': 'nombre_de_la_entidad',
        'nit_entidad': 'nit_de_la_entidad',
        'departamento': 'departamento_entidad',
        'municipio': 'municipio_entidad',
        'estado_proceso': 'estado_del_proceso',
        'modalidad': 'modalidad_de_contrataci_n',
        'objeto_a_contratar': 'objeto_a_contratar',
        'objeto_proceso': 'objeto_del_proceso',
        'numero_proceso': 'numero_de_proceso',
        'valor_contrato': 'valor_contrato',
        'nombre_contratista': 'nom_raz_social_contratista',
        'url_contrato': 'url_contrato',
        'origen': 'origen',
        'tipo_documento': 'tipo_documento_proveedor',
        'documento_proveedor': 'documento_proveedor',
        'fecha_firma': 'fecha_de_firma_del_contrato',
        'fecha_inicio': 'fecha_inicio_ejecuci_n',
        'fecha_fin': 'fecha_fin_ejecuci_n',
    }

    # Aplicar filtros simples
    for filtro_key, filtro_value in filtros.items():
        if filtro_value and filtro_key in mapeo_campos:
            campo_api = mapeo_campos[filtro_key]
            if 'objeto' in filtro_key:
                params[campo_api] = {'$like': f'%{filtro_value}%'}
            else:
                params[campo_api] = filtro_value

    # Construir filtros WHERE para rangos
    where_conditions = []

    if filtros.get('valor_minimo', 0) > 0:
        where_conditions.append(f"valor_contrato >= {filtros['valor_minimo']}")

    if filtros.get('valor_maximo', 0) > 0:
        where_conditions.append(f"valor_contrato <= {filtros['valor_maximo']}")

    if filtros.get('fecha_firma_desde'):
        where_conditions.append(f"fecha_de_firma_del_contrato >= '{filtros['fecha_firma_desde']}'")

    if filtros.get('fecha_firma_hasta'):
        where_conditions.append(f"fecha_de_firma_del_contrato <= '{filtros['fecha_firma_hasta']}'")

    if filtros.get('fecha_inicio_desde'):
        where_conditions.append(f"fecha_inicio_ejecuci_n >= '{filtros['fecha_inicio_desde']}'")

    if filtros.get('fecha_inicio_hasta'):
        where_conditions.append(f"fecha_inicio_ejecuci_n <= '{filtros['fecha_inicio_hasta']}'")

    if where_conditions:
        params['$where'] = ' AND '.join(where_conditions)

    # Realizar petición con reintentos
    max_intentos = 3
    for intento in range(max_intentos):
        try:
            response = requests.get(url, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()
                return pd.DataFrame(data) if data else pd.DataFrame()

            elif response.status_code == 429:  # Rate limit
                if intento < max_intentos - 1:
                    time.sleep(2 ** intento)
                    continue

            print(f"Error HTTP: {response.status_code}")
            return pd.DataFrame()

        except Exception as e:
            if intento < max_intentos - 1:
                time.sleep(2)
                continue
            print(f"Error: {e}")
            return pd.DataFrame()

    return pd.DataFrame()

def mostrar_columnas_disponibles():
    """Muestra las columnas disponibles en SECOP II"""
    df_muestra = buscar_contratos(limite=1)
    if not df_muestra.empty:
        print("Columnas disponibles en SECOP II:")
        for i, col in enumerate(df_muestra.columns, 1):
            print(f"{i:2d}. {col}")
    return df_muestra.columns.tolist() if not df_muestra.empty else []

def exportar_excel(df, prefijo="secop"):
    """Exporta DataFrame a Excel con timestamp"""
    if df.empty:
        print("Sin datos para exportar")
        return None

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo = f"{prefijo}_{timestamp}.xlsx"

    try:
        df.to_excel(archivo, index=False)
        return archivo
    except Exception as e:
        print(f"Error al exportar: {e}")
        return None

def resumen_busqueda(df):
    """Muestra resumen de resultados"""
    if df.empty:
        print("No se encontraron contratos")
        return

    print(f"Encontrados: {len(df)} contratos")

    # Mostrar primeras 10 columnas
    cols_mostrar = list(df.columns)[:10]
    if len(df.columns) > 10:
        cols_mostrar.append(f"... y {len(df.columns) - 10} más")
    print(f"Columnas: {cols_mostrar}")

    # Verificar objeto del contrato
    if 'objeto_a_contratar' in df.columns:
        objetos_validos = df['objeto_a_contratar'].notna().sum()
        objetos_definidos = df[df['objeto_a_contratar'] != 'NO DEFINIDO']['objeto_a_contratar'].notna().sum()
        print(f"Contratos con objeto definido: {objetos_definidos}/{len(df)} ({objetos_validos} válidos)")

In [2]:
# ==============================================================================
# BLOQUE DE EJECUCIÓN - MODIFICA AQUÍ LOS FILTROS
# ==============================================================================

# Variables de filtros
limite = 100
nivel_entidad = ""
codigo_entidad = ""
nombre_entidad = ""
nit_entidad = ""
departamento = "La Guajira"
municipio = ""
estado_proceso = ""
modalidad = ""
objeto_a_contratar = ""
objeto_proceso = ""
numero_proceso = ""
valor_contrato = ""
nombre_contratista = ""
url_contrato = ""
origen = ""
tipo_documento = ""
documento_proveedor = ""
valor_minimo = 0
valor_maximo = 0
fecha_firma_desde = ""
fecha_firma_hasta = ""
fecha_inicio_desde = ""
fecha_inicio_hasta = ""

# Ejecutar búsqueda
df = buscar_contratos(
    limite=limite,
    nivel_entidad=nivel_entidad,
    codigo_entidad=codigo_entidad,
    nombre_entidad=nombre_entidad,
    nit_entidad=nit_entidad,
    departamento=departamento,
    municipio=municipio,
    estado_proceso=estado_proceso,
    modalidad=modalidad,
    objeto_a_contratar=objeto_a_contratar,
    objeto_proceso=objeto_proceso,
    numero_proceso=numero_proceso,
    valor_contrato=valor_contrato,
    nombre_contratista=nombre_contratista,
    url_contrato=url_contrato,
    origen=origen,
    tipo_documento=tipo_documento,
    documento_proveedor=documento_proveedor,
    valor_minimo=valor_minimo,
    valor_maximo=valor_maximo,
    fecha_firma_desde=fecha_firma_desde,
    fecha_firma_hasta=fecha_firma_hasta,
    fecha_inicio_desde=fecha_inicio_desde,
    fecha_inicio_hasta=fecha_inicio_hasta
)

# Guardar resultados
if not df.empty:
    archivo = exportar_excel(df)
    if archivo:
        print(f"Guardado en: {archivo}")
    resumen_busqueda(df)
else:
    print("No se encontraron contratos")

# Análisis rápido
df


Guardado en: secop_20250729_125717.xlsx
Encontrados: 100 contratos
Columnas: ['nivel_entidad', 'codigo_entidad_en_secop', 'nombre_de_la_entidad', 'nit_de_la_entidad', 'departamento_entidad', 'municipio_entidad', 'estado_del_proceso', 'modalidad_de_contrataci_n', 'objeto_a_contratar', 'objeto_del_proceso', '... y 12 más']
Contratos con objeto definido: 99/100 (100 válidos)


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,...,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,TERRITORIAL,244847027,LA GUAJIRA - INSTITUCIÓN ETNOEDUCATIVA INTEGRA...,901167840,La Guajira,Uribia,Liquidado,Régimen Especial,COMPRA DE POLIZAS DE GARANTIA DE BUEN MANEJO D...,COMPRA DE PoLIZAS DE GARANTiA DE BUEN MANEJO D...,...,2020-01-17T00:00:00.000,2020-01-27T00:00:00.000,20-4-10732881,2020-001,714000,LA PREVISORA S.A,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Jurídica,860002400
1,Nacional,703458380,ICBF REGIONAL GUAJIRA,899999239,La Guajira,No Definido,Modificado,Contratación directa,PRESTAR SERVICIOS PROFESIONALES PARA LA DIRECC...,PRESTAR SERVICIOS PROFESIONALES PARA LA DIRECC...,...,2021-01-26T00:00:00.000,2021-12-31T00:00:00.000,CO1.PCCNTR.2169033,44000872021,42388667,SHIRLY JULIETH RADILLO GUARDIOLA,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,1118861807
2,Territorial,702424037,ALCALDIA DISTRITO DE RIOHACHA,892115007,La Guajira,Riohacha,Modificado,Contratación directa,PRESTACION DE \nSERVICIOS ASISTENCIALES DE APO...,PRESTACION DE \nSERVICIOS ASISTENCIALES DE APO...,...,2022-02-03T00:00:00.000,2022-12-09T00:00:00.000,CO1.PCCNTR.3557203,CPS-253-2022,22669064,ELSI ENITH PADILLA CANTILLO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,40936837
3,NACIONAL,122039000,INSTITUTO NACIONAL DE FORMACIÓN TÉCNICA PROFES...,860402193,La Guajira,San Juan del Cesar,Celebrado,Contratación Directa (Ley 1150 de 2007),PRESTA SERVICIOS PROFESIONALES JURIDICOS BRIND...,PRESTA SERVICIOS PROFESIONALES JURIDICOS BRIND...,...,2025-06-24T00:00:00.000,2025-12-06T00:00:00.000,25-12-14444892,CPS-141-2025,13337500,ELIANA ISABEL GONZALEZ SUAREZ,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Cédula de Ciudadanía,1122416173
4,Nacional,704156207,SENA REGIONAL GUAJIRA Grupo Mixto de Apoyo Adm...,899999034,La Guajira,Riohacha,En ejecución,Mínima cuantía,ADQUIRIR A TITULO DE COMPRAVENTA MATERIALES EQ...,ADQUIRIR A TITULO DE COMPRAVENTA MATERIALES EQ...,...,2024-04-01T00:00:00.000,2024-04-22T00:00:00.000,CO1.PCCNTR.6140413,CO1.PCCNTR.6140413,116221270,JP SOLUCIONES COLOMBIA SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901247893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TERRITORIAL,244874022,LA GUAJIRA - HOSPITAL SANTO TOMAS DE VILLANUEVA,800075650,La Guajira,Villanueva,Celebrado,Régimen Especial,SUMINISTRO DE LLANTAS ACEITES Y OTROS PARA EL...,SUMINISTRO DE LLANTAS ACEITES Y OTROS PARA EL...,...,2022-04-05T00:00:00.000,2022-10-05T00:00:00.000,22-4-13093007,020-2022,6000000,ANA ISABEL GARCIA ARRIETA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Natural,23084709
96,Nacional,704156207,SENA REGIONAL GUAJIRA Grupo Mixto de Apoyo Adm...,899999034,La Guajira,Riohacha,Cerrado,Contratación directa,Prestar los servicios profesionales de carácte...,Prestar los servicios profesionales de carácte...,...,2021-07-02T00:00:00.000,2021-12-18T00:00:00.000,CO1.PCCNTR.2631252,CO1.PCCNTR.2631252,17024644,DEYANETH IPUANA BRITO,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,26985733
97,TERRITORIAL,244874011,LA GUAJIRA - ALCALDÍA MUNICIPIO DE VILLANUEVA,892115198,La Guajira,Villanueva,Celebrado,Contratación Mínima Cuantía,CONSTITUCIoN DE UN ENCARGO FIDUCIARIO PARA LA ...,CONSTITUCIoN DE UN ENCARGO FIDUCIARIO PARA LA ...,...,2025-06-06T00:00:00.000,2026-01-06T00:00:00.000,25-13-14407667,PMC-019-2025,39850000,FIDUCIARIA POPULAR S.A,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Jurídica,800141235
98,Territorial,702424037,ALCALDIA DI

In [3]:
# Análisis de patrones en contratos SECOP
# Ejecutar después de obtener el DataFrame df

# Preparar datos
df['valor_contrato'] = pd.to_numeric(df['valor_contrato'], errors='coerce')

print("\n--- ANÁLISIS DE PATRONES ---\n")

# 1. CONTRATISTAS MÁS FRECUENTES (por documento único) - VERSIÓN MEJORADA
print("🔍 CONTRATISTAS MÁS FRECUENTES:")
print("-" * 60)

# Filtrar registros con documento válido (números reales, no textos genéricos)
docs_validos = df[
    df['documento_proveedor'].notna() &
    (df['documento_proveedor'] != '') &
    (df['documento_proveedor'] != 'NO DEFINIDO') &
    (df['documento_proveedor'].str.len() >= 7) &  # Mínimo 7 dígitos
    df['documento_proveedor'].str.isdigit()  # Solo números
].copy()

if not docs_validos.empty:
    # Agrupar por documento de proveedor
    analisis_por_doc = docs_validos.groupby('documento_proveedor').agg({
        'nom_raz_social_contratista': ['count', 'first'],
        'valor_contrato': 'sum'
    }).round(0)

    # Simplificar nombres de columnas
    analisis_por_doc.columns = ['num_contratos', 'nombre_principal', 'valor_total']

    # Mostrar top 10 contratistas por frecuencia
    top_contratistas = analisis_por_doc.sort_values('num_contratos', ascending=False).head(10)

    for i, (documento, row) in enumerate(top_contratistas.iterrows(), 1):
        print(f"{i:2d}. DOC: {documento}")
        print(f"    Nombre: {row['nombre_principal'][:50]}")
        print(f"    Contratos: {int(row['num_contratos'])}")
        print(f"    Valor total: ${row['valor_total']:,.0f}")

        # Verificar si hay variaciones en el nombre
        variaciones = docs_validos[docs_validos['documento_proveedor'] == documento]['nom_raz_social_contratista'].unique()
        if len(variaciones) > 1:
            print(f"    ⚠️  Variaciones de nombre ({len(variaciones)}):")
            for variacion in variaciones[:3]:
                print(f"       - {variacion[:45]}")
            if len(variaciones) > 3:
                print(f"       ... y {len(variaciones) - 3} más")

        # NUEVO: Mostrar URLs de contratos directamente aquí
        contratos_contratista = docs_validos[docs_validos['documento_proveedor'] == documento]
        urls_contratos = []

        # Revisar todas las posibles columnas de URL
        posibles_columnas_url = ['url_contrato', 'url_del_proceso', 'urlproceso', 'link_contrato']

        for col in posibles_columnas_url:
            if col in contratos_contratista.columns:
                urls_temp = contratos_contratista[col].dropna()
                urls_validas = [str(url).strip() for url in urls_temp if url and str(url).strip() != '' and str(url).lower() != 'nan']
                urls_contratos.extend(urls_validas)

        # Eliminar duplicados manteniendo orden
        urls_contratos = list(dict.fromkeys(urls_contratos))

        # Filtrar URLs que parecen válidas (contienen http o www)
        urls_activas = [url for url in urls_contratos if 'http' in url.lower() or 'www' in url.lower()]

        if urls_activas:
            print(f"    📋 URLs de contratos ({len(urls_activas)}):")
            for j, url in enumerate(urls_activas[:5], 1):
                print(f"       {j}. {url}")
            if len(urls_activas) > 5:
                print(f"       ... y {len(urls_activas) - 5} más")
        else:
            print(f"    📋 Sin URLs activas disponibles")

        print()

else:
    print("❌ No hay documentos válidos para analizar")

# 2. CALIDAD DE DATOS
print(f"\n🔍 CALIDAD DE DATOS:")
print("-" * 40)
total_registros = len(df)
sin_documento = len(df[df['documento_proveedor'].isna() | (df['documento_proveedor'] == '')])
no_definido = len(df[df['documento_proveedor'] == 'NO DEFINIDO'])
no_numerico = len(df[~df['documento_proveedor'].str.isdigit() | (df['documento_proveedor'].str.len() < 7)])
documentos_validos = len(docs_validos)
documentos_unicos = df['documento_proveedor'].nunique()
nombres_unicos = df['nom_raz_social_contratista'].nunique()

print(f"Total registros: {total_registros}")
print(f"Sin documento: {sin_documento} ({sin_documento/total_registros*100:.1f}%)")
print(f"'NO DEFINIDO': {no_definido} ({no_definido/total_registros*100:.1f}%)")
print(f"Documentos no numéricos o cortos: {no_numerico} ({no_numerico/total_registros*100:.1f}%)")
print(f"Documentos válidos para análisis: {documentos_validos} ({documentos_validos/total_registros*100:.1f}%)")
print(f"Documentos únicos totales: {documentos_unicos}")
print(f"Nombres únicos: {nombres_unicos}")
if documentos_validos > 0:
    docs_unicos_validos = docs_validos['documento_proveedor'].nunique()
    print(f"Documentos únicos válidos: {docs_unicos_validos}")
    print(f"Ratio doc válidos/nombres: {docs_unicos_validos/nombres_unicos:.2f}")

# 3. ESTADÍSTICAS DE VALORES
print(f"\n💰 ESTADÍSTICAS DE VALORES:")
print("-" * 40)
valores_validos = df[df['valor_contrato'].notna()]
if not valores_validos.empty:
    promedio = valores_validos['valor_contrato'].mean()
    mediana = valores_validos['valor_contrato'].median()
    sobre_promedio = valores_validos[valores_validos['valor_contrato'] > promedio]

    print(f"Valor promedio: ${promedio:,.0f}")
    print(f"Valor mediana: ${mediana:,.0f}")
    print(f"Contratos sobre promedio: {len(sobre_promedio)}/{len(valores_validos)} ({len(sobre_promedio)/len(valores_validos)*100:.1f}%)")

# 4. CONTRATOS DE MAYOR VALOR
print(f"\n🏆 CONTRATOS DE MAYOR VALOR:")
print("-" * 50)
valores_validos = df[df['valor_contrato'].notna()]
if not valores_validos.empty:
    top_valores = valores_validos.nlargest(5, 'valor_contrato')
    for i, (_, row) in enumerate(top_valores.iterrows(), 1):
        print(f"{i}. ${row['valor_contrato']:,.0f}")
        print(f"   Contratista: {row['nom_raz_social_contratista'][:40]}")
        print(f"   Entidad: {row['nombre_de_la_entidad'][:35]}")
        if 'documento_proveedor' in row and pd.notna(row['documento_proveedor']):
            print(f"   Documento: {row['documento_proveedor']}")
        print()

# 5. MODALIDADES DE CONTRATACIÓN
print("\n📋 MODALIDADES MÁS COMUNES:")
print("-" * 40)
if 'modalidad_de_contrataci_n' in df.columns:
    modalidades = df['modalidad_de_contrataci_n'].value_counts().head(5)
    for modalidad, freq in modalidades.items():
        pct = (freq / len(df)) * 100
        print(f"{modalidad}: {freq} ({pct:.1f}%)")

# 6. PREPARACIÓN PARA INVESTIGACIÓN WEB
def preparar_investigacion_web(num_top=3):
    """Prepara los datos para investigación web"""
    if docs_validos.empty:
        return []

    top_investigar = analisis_por_doc.sort_values('num_contratos', ascending=False).head(num_top)
    contratistas_investigar = []

    for i, (documento, row) in enumerate(top_investigar.iterrows(), 1):
        contratos_contratista = docs_validos[docs_validos['documento_proveedor'] == documento]
        todas_variaciones = contratos_contratista['nom_raz_social_contratista'].unique()
        nombre_mas_completo = max(todas_variaciones, key=len)

        # Obtener URLs
        urls_contratos = []
        for col in ['url_contrato', 'url_del_proceso', 'urlproceso', 'link_contrato']:
            if col in contratos_contratista.columns:
                urls_temp = contratos_contratista[col].dropna()
                urls_validas = [str(url).strip() for url in urls_temp if url and str(url).strip() != '' and str(url).lower() != 'nan']
                urls_contratos.extend(urls_validas)

        urls_activas = [url for url in list(dict.fromkeys(urls_contratos)) if 'http' in url.lower() or 'www' in url.lower()]

        contratista_info = {
            'ranking': i,
            'documento': documento,
            'nombre_principal': row['nombre_principal'],
            'nombre_completo': nombre_mas_completo,
            'num_contratos': int(row['num_contratos']),
            'valor_total': row['valor_total'],
            'variaciones_nombre': list(todas_variaciones),
            'urls_contratos': urls_activas
        }
        contratistas_investigar.append(contratista_info)

    return contratistas_investigar

# Ejecutar preparación (sin output visible)
contratistas_para_web = preparar_investigacion_web(3)


--- ANÁLISIS DE PATRONES ---

🔍 CONTRATISTAS MÁS FRECUENTES:
------------------------------------------------------------
 1. DOC: 901247893
    Nombre: JP SOLUCIONES COLOMBIA SAS
    Contratos: 2
    Valor total: $545,562,560
    📋 URLs de contratos (2):
       1. https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.5845649&isFromPublicArea=True&isModal=true&asPopupView=true
       2. https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.2127675&isFromPublicArea=True&isModal=true&asPopupView=true

 2. DOC: 84082962
    Nombre: RICARDO JAVIER REALES MAGDANIEL
    Contratos: 2
    Valor total: $25,400,000
    📋 URLs de contratos (2):
       1. https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.1884528&isFromPublicArea=True&isModal=true&asPopupView=true
       2. https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.4140009&isFromPublicArea=

In [4]:
# Investigación web de contratistas SECOP
# Ejecutar después del análisis de patrones

# Instalaciones
!pip install langchain langchain-openai tavily-python requests beautifulsoup4

from tavily import TavilyClient
from langchain_openai import ChatOpenAI
import requests
from bs4 import BeautifulSoup
from google.colab import userdata
import time

# ============================================================================
# CONFIGURACIÓN
# ============================================================================

NUM_CONTRATISTAS_TOP = 3  # Número de contratistas a investigar
MAX_FUENTES_WEB = 5       # Número de fuentes web por contratista

# Base de datos de conexiones para grafo
conexiones_encontradas = []

# Configuración APIs
TAVILY_KEY = userdata.get('TAVILY_KEY')
OPENAI_KEY = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o", temperature=0.1, api_key=OPENAI_KEY)
tavily = TavilyClient(api_key=TAVILY_KEY)

def buscar_web(query, max_results=None):
    """Busca información en web sobre el contratista"""
    if max_results is None:
        max_results = MAX_FUENTES_WEB

    try:
        response = tavily.search(
            query=query,
            max_results=max_results,
            topic="general",
            location="CO",
            language="es"
        )

        # Construir contenido con URLs
        contenido_con_fuentes = []
        urls_fuentes = []

        for r in response.get("results", []):
            titulo = r.get('title', '')
            contenido = r.get('content', '')[:400]
            url = r.get('url', '')

            contenido_con_fuentes.append(f"**{titulo}**\n{contenido}...\nFuente: {url}")
            urls_fuentes.append(url)

        return "\n\n".join(contenido_con_fuentes), urls_fuentes

    except Exception as e:
        return f"Error en búsqueda: {e}", []

def scrape_contrato_secop(url):
    """Extrae información básica de un contrato SECOP"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code != 200:
            return "No se pudo acceder al contrato"

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraer información relevante
        info = []

        # Buscar campos comunes en SECOP
        campos = ['objeto', 'modalidad', 'estado', 'valor', 'plazo', 'supervisor']
        for campo in campos:
            elementos = soup.find_all(text=lambda x: x and campo.lower() in x.lower())
            if elementos:
                info.append(f"{campo.title()}: {elementos[0][:100]}")

        return "\n".join(info) if info else "Información no disponible"

    except Exception as e:
        return f"Error al acceder: {str(e)}"

def investigar_contratista_por_documento(documento, nombre_busqueda, info_contratista):
    """Genera informe web del contratista basado en documento único"""

    # Datos del contratista en SECOP
    contratos_contratista = df[df['documento_proveedor'] == documento]
    num_contratos = len(contratos_contratista)
    valor_total = contratos_contratista['valor_contrato'].sum()

    # Obtener información adicional
    variaciones_nombre = contratos_contratista['nom_raz_social_contratista'].unique()
    entidades_contratantes = contratos_contratista['nombre_de_la_entidad'].unique()

    print(f"\n{'='*60}")
    print(f"🔍 INVESTIGANDO: {nombre_busqueda}")
    print(f"📄 DOCUMENTO: {documento}")
    print(f"{'='*60}")
    print(f"📊 Contratos en SECOP: {num_contratos}")
    print(f"💰 Valor total: ${valor_total:,.0f}")
    print(f"📝 Variaciones del nombre: {len(variaciones_nombre)}")

    # Mostrar variaciones si las hay
    if len(variaciones_nombre) > 1:
        print(f"   Nombres registrados:")
        for var in variaciones_nombre:
            freq = len(contratos_contratista[contratos_contratista['nom_raz_social_contratista'] == var])
            print(f"   - {var} ({freq} contratos)")

    # Información del contrato SECOP
    url_contrato_secop = ""
    info_contrato = ""
    if 'url_contrato' in contratos_contratista.columns:
        urls_validas = contratos_contratista['url_contrato'].dropna()
        if not urls_validas.empty:
            url_contrato_secop = urls_validas.iloc[0]
            print(f"🔗 Analizando contrato: {url_contrato_secop}")
            info_contrato = scrape_contrato_secop(url_contrato_secop)

    # Buscar información web
    print(f"🌐 Buscando información web...")
    informacion_web, urls_fuentes = buscar_web(f'"{nombre_busqueda}" Colombia NIT {documento}')

    # Búsqueda alternativa si no hay resultados
    if "Error en búsqueda" in informacion_web or len(informacion_web) < 100:
        print(f"🔄 Búsqueda alternativa...")
        informacion_web, urls_fuentes = buscar_web(f'"{nombre_busqueda}" empresa Colombia')

    # Generar informe estructurado
# Generar informe estructurado - VERSIÓN MEJORADA
    prompt = f"""
    Analiza la siguiente información sobre el contratista con documento {documento}:

    DATOS SECOP:
    - Documento proveedor: {documento}
    - Nombre principal: {nombre_busqueda}
    - Variaciones del nombre: {', '.join(variaciones_nombre)}
    - Número de contratos: {num_contratos}
    - Valor total contratado: ${valor_total:,.0f}
    - Entidades contratantes principales: {', '.join(entidades_contratantes[:5])}
    - URL contrato SECOP: {url_contrato_secop}

    INFORMACIÓN DEL CONTRATO:
    {info_contrato}

    INFORMACIÓN WEB:
    {informacion_web}

    Genera un informe estructurado con estilo narrativo y detallado:

    1. **RESUMEN EJECUTIVO**
    Redacta 4-5 líneas que capturen la esencia del contratista: qué tipo de empresa es, su nivel de actividad en contratación pública, su especialización y aspectos relevantes que lo caractericen en el ecosistema de contratación estatal.

    2. **IDENTIFICACIÓN DE LA EMPRESA**
    Desarrolla una descripción completa incluyendo:
       - Razón social principal y denominaciones comerciales identificadas
       - NIT/Documento: {documento}
       - Si existen variaciones en el nombre, explica detalladamente las diferencias encontradas y posibles razones (cambios de denominación social, errores de transcripción, abreviaciones, etc.)
       - Ubicación geográfica y antigüedad aproximada si está disponible

    3. **ACTIVIDAD ECONÓMICA PRINCIPAL**
    Proporciona un análisis profundo del sector económico en el que opera, tipos de productos o servicios que ofrece basándose en los objetos contractuales, su nicho de especialización dentro del mercado de contratación pública, y capacidades técnicas evidenciadas a través de sus contratos.

    4. **ANÁLISIS DE ACTIVIDAD CONTRACTUAL**
    Desarrolla un panorama completo de su desempeño en contratación pública:
       - Descripción narrativa de la frecuencia de contratación ({num_contratos} contratos)
       - Análisis de los rangos de valores que maneja habitualmente
       - Identificación y descripción de las entidades con las que más contrata
       - Patrones temporales y tipos de modalidades que más utiliza
       - Diversificación de su portafolio de clientes institucionales

    5. **PERSONAS Y ENTIDADES RELACIONADAS**
    Investiga exhaustivamente y presenta TODAS las personas y entidades vinculadas al contratista encontradas en la información web. Para cada persona o entidad identificada proporciona una mini-descripción:
       - Nombres completos de personas mencionadas (representantes, socios, directores, etc.) con descripción de su rol y responsabilidades
       - Cargos o roles específicos y cualquier información sobre su trayectoria
       - Empresas o entidades asociadas con tipo de relación (subsidiaria, matriz, aliado, proveedor) y descripción de la conexión
       - Cualquier conexión empresarial o familiar mencionada con contexto
       - Organismos gremiales, cámaras de comercio o asociaciones

    Para cada persona/entidad encontrada, usa el formato:
    - Nombre completo | Cargo/rol | Mini-descripción de la relación con la empresa

    IMPORTANTE: Después de cada sección principal, incluye inmediatamente:
    Fuente: URL_ESPECÍFICA

    Utiliza un lenguaje profesional pero fluido, integrando la información de manera coherente. Donde URL_ESPECÍFICA es la URL completa que respalda esa información.
    NO uses asteriscos ni paréntesis alrededor de las URLs.
    """

    response = llm.invoke(prompt)

    # Extraer conexiones para el grafo
    prompt_conexiones = f"""
    Del siguiente informe sobre el contratista con documento {documento}, extrae únicamente:

    {response.content}

    Lista SOLO los nombres de personas y entidades mencionadas en formato:
    PERSONA: nombre completo | cargo/rol | documento base: {documento}
    ENTIDAD: nombre entidad | relación | documento base: {documento}

    Responde únicamente con la lista, sin explicaciones adicionales.
    """

    conexiones_response = llm.invoke(prompt_conexiones)

    # Guardar conexiones
    conexion_data = {
        'documento_principal': documento,
        'contratista_principal': nombre_busqueda,
        'variaciones_nombre': list(variaciones_nombre),
        'conexiones_texto': conexiones_response.content,
        'valor_total': valor_total,
        'num_contratos': num_contratos,
        'entidades_contratantes': list(entidades_contratantes)
    }
    conexiones_encontradas.append(conexion_data)

    # Construir informe completo con fuentes
    informe_completo = response.content

    informe_completo += "\n\n" + "="*60
    informe_completo += "\n🔗 FUENTES CONSULTADAS:"
    informe_completo += "\n" + "="*60

    if url_contrato_secop:
        informe_completo += f"\n📋 SECOP: {url_contrato_secop}"

    for i, url in enumerate(urls_fuentes, 1):
        informe_completo += f"\n🌐 Fuente {i}: {url}"

    informe_completo += f"\n\n📄 DOCUMENTO PROVEEDOR: {documento}"
    informe_completo += f"\n📊 TOTAL CONTRATOS ANALIZADOS: {num_contratos}"

    return informe_completo

def ejecutar_investigacion_automatica():
    """Investiga automáticamente los top contratistas"""

    if not contratistas_para_web:
        print("❌ No hay contratistas preparados para investigación")
        return

    print("="*70)
    print(f"🔍 INVESTIGACIÓN WEB AUTOMÁTICA")
    print("="*70)

    for contratista_info in contratistas_para_web:
        documento = contratista_info['documento']
        nombre_busqueda = contratista_info['nombre_completo']

        informe = investigar_contratista_por_documento(documento, nombre_busqueda, contratista_info)
        print(informe)

        if contratista_info != contratistas_para_web[-1]:
            print("\n" + "🔄 SIGUIENTE CONTRATISTA..." + "\n")
            time.sleep(2)  # Pausa entre búsquedas

    print("\n" + "="*70)
    print("✅ INVESTIGACIÓN AUTOMÁTICA COMPLETADA")
    print("="*70)

def ejecutar_investigacion_manual(documento_proveedor):
    """Investiga un contratista específico por su documento"""

    # Buscar el contratista por documento
    contratos_encontrados = df[df['documento_proveedor'] == documento_proveedor]

    if contratos_encontrados.empty:
        print(f"❌ No se encontraron contratos para el documento: {documento_proveedor}")
        return

    # Obtener el nombre más completo
    nombres = contratos_encontrados['nom_raz_social_contratista'].unique()
    nombre_busqueda = max(nombres, key=len)

    print("\n" + "="*70)
    print("🔍 INVESTIGACIÓN MANUAL")
    print("="*70)

    # Crear info del contratista
    contratista_info = {
        'documento': documento_proveedor,
        'nombre_completo': nombre_busqueda,
        'num_contratos': len(contratos_encontrados),
        'valor_total': contratos_encontrados['valor_contrato'].sum()
    }

    informe = investigar_contratista_por_documento(documento_proveedor, nombre_busqueda, contratista_info)
    print(informe)

    print("\n" + "="*70)
    print("✅ INVESTIGACIÓN MANUAL COMPLETADA")
    print("="*70)

# ============================================================================
# EJECUCIÓN
# ============================================================================

# Ejecutar investigación automática
ejecutar_investigacion_automatica()

# ============================================================================
# MOSTRAR CONEXIONES ENCONTRADAS
# ============================================================================

print("\n" + "="*70)
print("🔗 CONEXIONES ENCONTRADAS PARA GRAFO")
print("="*70)

for i, conexion in enumerate(conexiones_encontradas, 1):
    print(f"\n{i}. {conexion['contratista_principal']} (DOC: {conexion['documento_principal']}):")
    print(f"   Contratos: {conexion['num_contratos']} | Valor: ${conexion['valor_total']:,.0f}")
    print(f"   Conexiones:\n{conexion['conexiones_texto']}")
    print("-" * 50)

# ============================================================================
# INVESTIGACIÓN MANUAL (OPCIONAL)
# ============================================================================

# Para investigar un contratista específico, usar:
# ejecutar_investigacion_manual("NUMERO_DE_DOCUMENTO_AQUI")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.7 MB/s eta 0:00:00
🔍 INVESTIGACIÓN WEB AUTOMÁTICA

🔍 INVESTIGANDO: JP SOLUCIONES COLOMBIA SAS
📄 DOCUMENTO: 901247893
📊 Contratos en SECOP: 2
💰 Valor total: $545,562,560
📝 Variaciones del nombre: 1
🔗 Analizando contrato: https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.5845649&isFromPublicArea=True&isModal=true&asPopupView=true


/tmp/ipython-input-4-430900134.py:79: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elementos = soup.find_all(text=lambda x: x and campo.lower() in x.lower())


🌐 Buscando información web...
1. **RESUMEN EJECUTIVO**

JP Soluciones Colombia SAS es una empresa que se desempeña en el ámbito de la contratación pública en Colombia, destacándose por su participación en proyectos con entidades gubernamentales. Con un total de dos contratos adjudicados, la empresa ha logrado un valor total contratado de $545,562,560, lo que refleja su capacidad para manejar proyectos de envergadura. Su principal cliente es el SENA Regional Guajira, lo que sugiere una especialización en servicios que apoyan la gestión administrativa y operativa de entidades estatales.

Fuente: https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.5845649&isFromPublicArea=True&isModal=true&asPopupView=true

2. **IDENTIFICACIÓN DE LA EMPRESA**

JP Soluciones Colombia SAS es la razón social principal de la empresa, sin variaciones significativas en su denominación comercial. El NIT de la empresa es 901247893. No se han encontrado diferencias en el nombr

/tmp/ipython-input-4-430900134.py:79: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elementos = soup.find_all(text=lambda x: x and campo.lower() in x.lower())


🌐 Buscando información web...
**1. RESUMEN EJECUTIVO**

Ricardo Javier Reales Magdaniel es un contratista individual que ha participado en el ecosistema de contratación pública en Colombia, específicamente con la Alcaldía del Distrito de Riohacha. Con un total de dos contratos adjudicados, su actividad en el sector público es moderada, reflejando un enfoque posiblemente especializado en servicios o productos específicos que no se detallan en la información disponible. El valor total de sus contratos asciende a $25,400,000, lo que sugiere un nivel de operación en proyectos de pequeña a mediana escala.

Fuente: https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.1884528&isFromPublicArea=True&isModal=true&asPopupView=true

**2. IDENTIFICACIÓN DE LA EMPRESA**

La razón social principal del contratista es Ricardo Javier Reales Magdaniel, sin variaciones significativas en su denominación comercial. El documento de identificación asociado es 84082962. No 

/tmp/ipython-input-4-430900134.py:79: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elementos = soup.find_all(text=lambda x: x and campo.lower() in x.lower())


🌐 Buscando información web...
**1. RESUMEN EJECUTIVO**

Franklin Almanza, identificado con el documento 1065563151, es un contratista que ha incursionado en el ecosistema de contratación pública en Colombia. Con un contrato registrado por un valor total de $41,980,000, su actividad se centra en la colaboración con entidades gubernamentales, específicamente con la Gobernación de La Guajira. Aunque su participación en el mercado de contratación pública es limitada en términos de cantidad de contratos, el valor del contrato sugiere un enfoque en proyectos de mediana escala. La información disponible no proporciona detalles sobre su especialización, pero su relación con una entidad gubernamental indica un posible enfoque en servicios o productos que apoyan la gestión pública.

Fuente: https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.1973770&isFromPublicArea=True&isModal=true&asPopupView=true

**2. IDENTIFICACIÓN DE LA EMPRESA**

Franklin Almanza es 